In [1]:
import pandas as pd
import math
import numpy as np
import itertools

In [2]:
#statsDF - initial stats dataframe
#gamesDF - initial games dataframe
#newGamesDF - formatted games dataframe with game date as index and rest of record in one row
#finalStats - formatted stats dataframe with player's name as the index
#mainDF: The dataframe where the main analysis will take place.

In [67]:
stats = '/home/acer/github/nba/stats.csv'
games = '/home/acer/github/nba/games.csv'

In [92]:
#Dump the stats dataframe into a csv and clean it up
cols = ['PLAYER', 'MIN', 'FGM','FGA', 'FG%', '3PM', '3PA','3P%','FTM','FTA','FT%','OREB','DREB','REB','AST','TOV','STL','BLK','PF','PTS','+/-']
statsDF = pd.read_csv(stats, header= None, names=cols, index_col=False, error_bad_lines=False, delim_whitespace=True)
statsDF[cols] = statsDF[cols].replace({'\$': '', ',': ''}, regex=True)
statsDF['PLAYER'] = statsDF['PLAYER'].str.replace('.','')
statsDF['PLAYER'] = statsDF['PLAYER'].str.replace('-','')
statsDF['PLAYER'] = statsDF['PLAYER'].str.replace("'",'')
statsDF['MIN'] = statsDF['MIN'].str.replace("'",'')
statsDF['MIN'] = statsDF['MIN'].replace({'\$': '', '-': ''}, regex=True)
#Drop the first line for the team name
statsDF.drop(0,inplace=True)

In [93]:
#The nba.com site changed its formatting so this is to remove non-existent rows

new_format_mask = 0
for index, row in statsDF.iterrows():
    if new_format_mask == 1:
        statsDF.drop(index, inplace=True)
        if row.iloc[0] == 'Totals:' and math.isnan(row.iloc[1]) == True:
            new_format_mask = 0 
        else:
            continue
    else:
        if row.iloc[0] == 'PLAYER' and row.iloc[1] != "MIN":
            new_format_mask += 1
        else:
            continue

for index, row in statsDF.iterrows():
    if row.iloc[0][3:4] == ":":
        statsDF.drop(index, inplace=True)

    

In [94]:
statsDF = statsDF[statsDF.PLAYER != 'PLAYER']
statsDF = statsDF[statsDF.PLAYER != 'Totals:']
statsDF = statsDF[statsDF.PLAYER != 'INACTIVE']
statsDF = statsDF.reset_index(drop=True)

In [95]:
for index, row in statsDF.iterrows():
    try: 
        if row.iloc[1].isalpha() and statsDF.iloc[index + 1][1].isalpha():
            home_team_length = index/2
            statsDF.drop(index, inplace=True)
            statsDF = statsDF.reset_index(drop=True)
            break
    except:
        continue

In [96]:
#Setting up the index on the main stats dataframe
mainDFIndex = []
for index, row in statsDF.iterrows():
    if row.iloc[0] == 'Nene':
        mainDFIndex.append(str(row['PLAYER']) + " " + str(row['MIN']))
    elif row.iloc[0].isalpha() == True and row.iloc[1].isalpha() == True:
        mainDFIndex.append(str(row['PLAYER']) + " " + str(row['MIN']))
print(mainDFIndex)

['Justin Jackson', 'Zach Randolph', 'Willie CauleyStein', 'Buddy Hield', 'George Hill', 'Skal Labissiere', 'DeAaron Fox', 'Garrett Temple', 'Kosta Koufos', 'Vince Carter', 'Frank Mason', 'Malachi Richardson', 'JaKarr Sampson', 'Wilson Chandler', 'Paul Millsap', 'Nikola Jokic', 'Gary Harris', 'Jamal Murray', 'Will Barton', 'Mason Plumlee', 'Emmanuel Mudiay', 'Kenneth Faried', 'Juancho Hernangomez', 'Trey Lyles', 'Malik Beasley', 'Darrell Arthur']


In [97]:
#Setup dicts from the DF
statsDF_Dict = statsDF.T.to_dict().values()

In [100]:
dict_list = []
#Iterate through the statsDF and only return lines with stats or lines where players sat
for index, i in enumerate(statsDF_Dict):
    if index == 1 or index % 2 != 0:
        dict_list.append(i)
    

In [101]:
finalStats = pd.DataFrame(dict_list)
finalStats.index = mainDFIndex
finalStats.drop(['+/-'], axis=1, inplace=True)
finalStats = finalStats.rename(columns={'PLAYER':'MIN','MIN':'FGM', 'FGM':'FGA', 'FGA':'FG%', 'FG%':'3PM', '3PM':'3PA', '3PA':'3P%', '3P%':'FTM', 'FTM':'FTA', 'FTA':'FT%', 'FT%':'OREB', 'OREB':'DREB', 'DREB':'REB', 'REB':'AST', 'AST':'TOV', 'TOV':'BLK', 'BLK':'PF', 'PF':'PTS', 'PTS':'+/-'})
#This will replace the nan's with DNP
finalStats = finalStats.fillna('NA')

In [77]:
finalStats

,+/-,3P%,3PA,3PM,AST,BLK,DREB,FG%,FGA,FGM,...,FTA,FTM,MIN,OREB,PF,PLAYER,PTS,REB,STL,TOV
0,NaN,0,33.3,3,2,2,1,1,40.0,5,...,0.0,0,2,1,5,19:36,-8,0,0,0
1,NaN,0,50.0,2,2,3,9,1,38.5,13,...,0.0,0,5,5,11,20:01,-13,0,0,2
2,NaN,0,0.0,0,0,2,8,0,20.0,5,...,0.0,2,1,7,2,26:32,-12,1,0,0
3,NaN,1,0.0,4,0,2,3,0,10.0,10,...,100,1,1,3,3,22:16,-10,3,0,0
4,NaN,2,0.0,2,3,0,0,0,20.0,5,...,100,2,1,0,4,22:21,-8,1,0,0
5,NaN,0,50.0,2,1,3,4,1,50.0,12,...,0.0,0,6,2,13,25:01,5,4,1,2
6,NaN,1,100,1,2,4,5,1,61.5,13,...,100,1,8,3,18,28:51,-6,1,0,0
7,NaN,1,33.3,3,2,1,1,1,33.3,3,...,50.0,2,1,1,4,16:46,-11,2,1,1
8,NaN,0,0.0,0,0,1,3,0,100,1,...,0.0,0,1,2,2,15:32,-4,1,1,1
9,NaN,0,0.0,1,0,0,1,0,0.0,2,...,0.0,0,0,1,0,5:33,-3,0,0,0


In [78]:
#GAMES CELL: This prepares the games section into a dataframe
#Passing in csv file and getting rid of the unused rows
gamesDF = pd.read_csv(games, header= None, names=['a','b','c','d','e','f','g','h','i'], index_col=False, error_bad_lines=False, delim_whitespace=True)
gamesDF.columns = [col.replace(',', '') for col in gamesDF.columns]
#if you want to operate on multiple columns, put them in a list like so:
games_cols = ['a','b','c','d','e','f','g','h','i']
# pass them to df.replace(), specifying each char and it's replacement:
gamesDF[games_cols] = gamesDF[games_cols].replace({'\$': '', ',': ''}, regex=True)
# This will add the winner to column b of each 26th row
for index, row in gamesDF.iterrows():
    if index % 26 == 0 or index == 0:
        if int(gamesDF['a'].iloc[index+4]) > int(gamesDF['a'].iloc[index + 8]):
            gamesDF['b'].iloc[index] = gamesDF['a'].iloc[index + 1]
        else:
            gamesDF['b'].iloc[index] = gamesDF['a'].iloc[index + 5]
    else:
        pass

In [79]:
#This cell will arrange the gamesDF into the actual formatted games dataframe, newGamesDF
#Setting up the index on the main stats dataframe
dateIndex = []
month = ['OCT', 'NOV', 'DEC', 'JAN', 'FEB']
for index, row in gamesDF.iterrows():
    for i in month:
        if gamesDF.iloc[index][0] == i:
            dateIndex.append(gamesDF.iloc[9,0] + " " + gamesDF.iloc[9,1][1] + ", " + gamesDF.iloc[9,2])
#The next lines will pull the data from the cells in the gamesDF and into the newGamesDF dataframe
gamesData = []
games = 0
for index, row in gamesDF.iterrows():
    if index == 0 or index % 26 == 0:
        gamesData.append(gamesDF.iloc[index+11,0])
        gamesData.append(gamesDF.iloc[index+12,0])
        gamesData.append(gamesDF.iloc[index,1])
        gamesData.append(gamesDF.iloc[index+11,1])
        gamesData.append(gamesDF.iloc[index+11,2])
        gamesData.append(gamesDF.iloc[index+11,3])
        gamesData.append(gamesDF.iloc[index+11,4])
        gamesData.append(gamesDF.iloc[index+12,1])
        gamesData.append(gamesDF.iloc[index+12,2])
        gamesData.append(gamesDF.iloc[index+12,3])
        gamesData.append(gamesDF.iloc[index+12,4])
        gamesData.append(gamesDF.iloc[index+8,0])
        gamesData.append(gamesDF.iloc[index+4,0])
        gamesData.append(gamesDF.iloc[index+22,2])
        gamesData.append(gamesDF.iloc[index+22,4])
        gamesData.append(gamesDF.iloc[index+22,6])
        games += 1
        
gamesData = np.array(gamesData)
gamesData = gamesData.reshape(games,16)
#Create an empty dataframe, based on the games date as the index
newGamesDF = pd.DataFrame(data = gamesData, columns=['Home','Away','W/L','1st Qtr H','2nd Qtr H','3rd Qtr H','4th Qtr H','1st Qtr A','2nd Qtr A','3rd Qtr A','4th Qtr A','Total H','Total A', 'Ref1','Ref2','Ref3'])
newGamesDF

,Home,Away,W/L,1st Qtr H,2nd Qtr H,3rd Qtr H,4th Qtr H,1st Qtr A,2nd Qtr A,3rd Qtr A,4th Qtr A,Total H,Total A,Ref1,Ref2,Ref3
0,SAC,DEN,DENVER,18,16,18,27,17,25,24,30,96,79,"Callahan""""","Ayotte""""",Ransom


In [80]:
for index,row in finalStats.iterrows():
    finalStats['1st Qtr H'] = newGamesDF['1st Qtr H'][0]
    finalStats['2nd Qtr H'] = newGamesDF['2nd Qtr H'][0]
    finalStats['3rd Qtr H'] = newGamesDF['3rd Qtr H'][0]
    finalStats['4th Qtr H'] = newGamesDF['4th Qtr H'][0]
    finalStats['1st Qtr A'] = newGamesDF['1st Qtr A'][0]
    finalStats['2nd Qtr A'] = newGamesDF['2nd Qtr A'][0]
    finalStats['3rd Qtr A'] = newGamesDF['3rd Qtr A'][0]
    finalStats['4th Qtr A'] = newGamesDF['4th Qtr A'][0]
    finalStats['Total H'] = newGamesDF['Total H'][0]
    finalStats['Total A'] = newGamesDF['Total A'][0]
    finalStats['Ref1'] = newGamesDF['Ref1'][0]
    finalStats['Ref2'] = newGamesDF['Ref2'][0]
    finalStats['Ref3'] = newGamesDF['Ref3'][0]
    finalStats['W/L'] = newGamesDF['W/L'][0]
    finalStats['Home'] = newGamesDF['Home'][0]
    finalStats['Away'] = newGamesDF['Away'][0]
    

In [81]:
#This function will calculate the projected fantasy points per game
def fantasy_points(row):
    try:
        score = (int(row['3PM']) * 3) + (int(row['FGM']) * 2) + (int(row['REB']) * 1) + (int(row['AST']) * 1.5) + (int(row['BLK']) * 3) + (int(row['STL']) * 3) + (int(row['TOV']) * -1)
        return score
    except:
        pass

finalStats['Fantasy Score'] = finalStats.apply(fantasy_points, axis=1)
finalStats = finalStats.fillna('NA')

In [82]:
finalStats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 38 columns):
+/-              26 non-null object
3P%              26 non-null object
3PA              26 non-null object
3PM              26 non-null object
AST              26 non-null object
BLK              26 non-null object
DREB             26 non-null object
FG%              26 non-null object
FGA              26 non-null object
FGM              26 non-null object
FT%              26 non-null object
FTA              26 non-null object
FTM              26 non-null object
MIN              26 non-null object
OREB             26 non-null object
PF               26 non-null object
PLAYER           26 non-null object
PTS              26 non-null object
REB              26 non-null object
STL              26 non-null object
TOV              26 non-null object
1st Qtr H        26 non-null object
2nd Qtr H        26 non-null object
3rd Qtr H        26 non-null object
4th Qtr H        26 non-null obje